In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import os
# path = ""
# path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
# os.chdir(path)
# import time
# FOLDERNAME = "LAW-OpenNMT-" + str(time.strftime("%Y%m%d-%H%M"))
# !mkdir $FOLDERNAME

# path = path + FOLDERNAME
# os.chdir(path)
# !pwd

import os
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/TED-OpenNMT-20210411-0138'
os.chdir(path)
!pwd

/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/TED-OpenNMT-20210411-0138


In [3]:
!nvidia-smi

Sun Apr 11 13:42:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 12.9MB/s 
     |████████████████████████████████| 2.6MB 27.1MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=cb95aa2e1bde7e7bb24ce427e808f71acbd7a246a80160e3ea1b05a0e6484872
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 79.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/law_bert.tar.gz
!mkdir data_bin
!tar -xvf 'law_bert.tar.gz'

--2021-04-11 01:38:49--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/law_bert.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4494433 (4.3M) [application/octet-stream]
Saving to: ‘law_bert.tar.gz’

law_bert.tar.gz     100%[===================>]   4.29M  --.-KB/s    in 0.1s    

2021-04-11 01:38:49 (29.6 MB/s) - ‘law_bert.tar.gz’ saved [4494433/4494433]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [6]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-11 01:38:53,812 INFO] Extracting features...
[2021-04-11 01:38:53,815 INFO]  * number of source features: 0.
[2021-04-11 01:38:53,815 INFO]  * number of target features: 0.
[2021-04-11 01:38:53,815 INFO] Building `Fields` object...
[2021-04-11 01:38:53,816 INFO] Building & saving training data...
[2021-04-11 01:38:53,861 INFO] Building shard 0.
[2021-04-11 01:38:54,533 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-11 01:38:55,138 INFO]  * tgt vocab size: 5128.
[2021-04-11 01:38:55,155 INFO]  * src vocab size: 13047.
[2021-04-11 01:38:55,248 INFO] Building & saving validation data...
[2021-04-11 01:38:55,488 INFO] Building shard 0.
[2021-04-11 01:38:55,530 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-11 01:38:57,771 INFO]  * src vocab size = 13047
[2021-04-11 01:38:57,772 INFO]  * tgt vocab size = 5128
[2021-04-11 01:38:57,772 INFO] Building model...
[2021-04-11 01:39:04,749 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(13047, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_featu

In [8]:
!ls -al model model/

model:
total 20340465
-rw------- 1 root root 694287743 Apr 11 02:59 en-vi_step_10000.pt
-rw------- 1 root root 694287743 Apr 11 01:47 en-vi_step_1000.pt
-rw------- 1 root root 694287743 Apr 11 03:07 en-vi_step_11000.pt
-rw------- 1 root root 694287743 Apr 11 03:15 en-vi_step_12000.pt
-rw------- 1 root root 694287743 Apr 11 03:23 en-vi_step_13000.pt
-rw------- 1 root root 694287743 Apr 11 03:31 en-vi_step_14000.pt
-rw------- 1 root root 694287743 Apr 11 03:39 en-vi_step_15000.pt
-rw------- 1 root root 694287743 Apr 11 03:47 en-vi_step_16000.pt
-rw------- 1 root root 694287743 Apr 11 03:55 en-vi_step_17000.pt
-rw------- 1 root root 694287743 Apr 11 04:03 en-vi_step_18000.pt
-rw------- 1 root root 694287743 Apr 11 04:11 en-vi_step_19000.pt
-rw------- 1 root root 694287743 Apr 11 04:19 en-vi_step_20000.pt
-rw------- 1 root root 694287743 Apr 11 01:55 en-vi_step_2000.pt
-rw------- 1 root root 694287743 Apr 11 04:27 en-vi_step_21000.pt
-rw------- 1 root root 694287743 Apr 11 04:35 en-vi_step

In [5]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-11 13:43:11,486 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [135], which does not match the required output shape [27, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an o

In [6]:
!tail vi_test

Việc giải quyết tranh chấp lao động do cơ quan, tổ chức, cá nhân có thẩm quyền giải quyết tranh chấp lao động tiến hành sau khi một trong hai bên có đơn yêu cầu do một trong hai bên từ chối thương lượng, thương lượng nhưng không thành hoặc thương lượng thành nhưng một trong hai bên không thực hiện.
Việc giám định do cá nhân hoặc do tập thể thực hiện.Giám định bổ sung
Bầu, miễn nhiệm, bãi nhiệm Chủ tịch Hội đồng thành viên; quyết định bổ nhiệm, miễn nhiệm, bãi nhiệm, ký và chấm dứt hợp đồng đối với Giám đốc hoặc Tổng giám đốc, Kế toán trưởng và người quản lý khác quy định tại Điều lệ công ty;
Người yêu cầu Tòa án Việt Nam công nhận và cho thi hành hoặc không công nhận tại Việt Nam bản án, quyết định dân sự của Tòa án nước ngoài; công nhận và cho thi hành phán quyết của Trọng tài nước ngoài phải nộp lệ phí theo quy định của pháp luật Việt Nam.
Họ, tên Thẩm phán, Thư ký phiên tòa, Kiểm sát viên; họ, tên Thẩm phán, Kiểm sát viên dự khuyết (nếu có);
Giới hạn trách nhiệm đối với người cứu hộ

In [7]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

fatal: destination path 'OpenNMT-py' already exists and is not an empty directory.


In [8]:
!ls -al

total 22476
drwx------  2 root root    4096 Apr 11 01:38 data_bin
-rw-------  1 root root  257314 Apr 11 01:32 en_test
-rw-------  1 root root 2096540 Apr 11 01:32 en_train
-rw-------  1 root root 2564912 Apr 11 01:35 en_train_EM_0.8
-rw-------  1 root root 2224075 Apr 11 01:35 en_train_EM_0.85
-rw-------  1 root root 2137239 Apr 11 01:35 en_train_EM_0.9
-rw-------  1 root root 2108457 Apr 11 01:35 en_train_EM_0.95
-rw-------  1 root root  801412 Apr 11 01:35 en_train_EM_factor_0.8
-rw-------  1 root root  688124 Apr 11 01:35 en_train_EM_factor_0.85
-rw-------  1 root root  659440 Apr 11 01:35 en_train_EM_factor_0.9
-rw-------  1 root root  650068 Apr 11 01:35 en_train_EM_factor_0.95
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.8
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.85
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.9
-rw-------  1 root root  148555 Apr 11 01:35 en_train_EM_score_0.95
-rw-------  1 root root  265528 Apr 1

In [9]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 39.76, 68.9/55.6/46.0/39.4 (BP=0.774, ratio=0.796, hyp_len=40853, ref_len=51293)
